In [3]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
import time
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot

In [4]:
df = pd.read_csv("train.csv")

In [5]:
df = df.drop('id', axis=1)

In [6]:
cols = list(df.columns)
target = df[cols[-1]]
target.unique()

array(['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9'], dtype=object)

In [7]:
x = df[cols[:-1]].values  # get all the columns data in a form of numpy array
x.shape  # data has 61878 rows and 94 columns
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, columns=cols[:-1])
X = df.values

In [8]:
le = LabelEncoder()
le = le.fit(target)
labels = le.transform(target)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [10]:
param_grid = {
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.0001, 0.001, 0.01, 0.1],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100, 200, 300, 400, 500]
        }

In [ ]:
model = XGBClassifier()
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=1, cv=kfold)
start = time.time()
grid_result = grid_search.fit(X_train, y_train)
stop = time.time()
print("Time Elapsed in training: {}".format(str(stop - start)))

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))